In [3]:
# Splitting text data and storing them in a list (of articles)
import io
docs = io.open(r"C:\Users\CHANDANA REDDY\Desktop\input-1.txt", mode="r", encoding="utf-8", errors="ignore").read().split('\n')  # list of strings
titles = [docs[i] for i in range(len(docs)) if i % 2 == 0] # list of string titles
contents = [docs[i] for i in range(len(docs)) if i % 2 == 1] # list of string contents
#print(contents[500])

In [4]:
# Preprocessing/ cleaning the data
import re
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize

# remove text between parenthesis
# contents = list(map(lambda x: re.sub(r"\(.*\)","",x), contents))

# remove all digits from text
contents = list(map(lambda x: re.sub(r"\d+","",x), contents))

stop = set(stopwords.words('english')) # set of stopwords
lemma = WordNetLemmatizer()
def clean(doc):
    # remove stopwords and words that are too short
    return [lemma.lemmatize(i, 'v') for i in word_tokenize(doc) if i not in stop and len(i) > 2]
cleaned = [clean(page.lower()) for page in contents]

print(cleaned[2])

['concept', 'angle', 'line', 'plane', 'pair', 'two', 'line', 'two', 'plan', 'line', 'plane', 'space', 'generalise', 'arbitrary', 'dimension', 'generalisation', 'first', 'discuss', 'jordan', 'pair', 'flats', 'euclidean', 'space', 'arbitrary', 'dimension', 'one', 'define', 'set', 'mutual', 'angle', 'invariant', 'isometric', 'transformation', 'euclidean', 'space', 'flats', 'intersect', 'shortest', 'distance', 'one', 'invariant', 'angle', 'call', 'canonical', 'principal', 'concept', 'angle', 'generalise', 'pair', 'flats', 'finite-dimensional', 'inner', 'product', 'space', 'complex', 'numbers.andbeing', 'relate', 'canonical', 'coordinate', 'basic', 'vectors', 'may', 'call', 'canonical.the', 'principal', 'angle', 'vectors', 'define', 'recursively', 'viaif', 'largest', 'angle', 'zero', 'one', 'subspace', 'subset', 'other.if', 'smallest', 'angle', 'zero', 'subspaces', 'intersect', 'least', 'line.the', 'number', 'angle', 'equal', 'zero', 'dimension', 'space', 'two', 'subspaces', 'intersect']


In [5]:
# Building word dicitonary
from gensim import corpora
# create the term dictionary of our corpus; terms are unique; each term is assigned an index
dictionary = corpora.Dictionary(cleaned)
print(dictionary)
dictionary.filter_extremes(no_below=3, no_above=0.7)
print(dictionary)
stoplist = set('also use make people know many call include part find become like mean often different usually take wikt come give well get since type list say change see refer actually iii aisne kinds pas ask would way something need things want every str'.split())
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
dictionary.filter_tokens(stop_ids)
print(dictionary)
dictionary.filter_n_most_frequent(50)
print(dictionary)

# This saves the dictionary to the local disk
dictionary.save_as_text('./dictionary.txt')


Dictionary(25733 unique tokens: ['abelian', 'addition', 'also', 'argument', 'article']...)
Dictionary(6621 unique tokens: ['abelian', 'addition', 'also', 'argument', 'article']...)
Dictionary(6583 unique tokens: ['abelian', 'addition', 'argument', 'article', 'attribution/share-alike']...)
Dictionary(6533 unique tokens: ['abelian', 'addition', 'argument', 'article', 'attribution/share-alike']...)


In [6]:
# Creating document-term matrix from vocabulary (dictionary)
doc_term_matrix = [dictionary.doc2bow(doc) for doc in cleaned]
print(len(doc_term_matrix))
print(doc_term_matrix[693])

694
[(1, 1), (7, 1), (11, 1), (14, 1), (22, 2), (24, 2), (30, 3), (34, 29), (39, 1), (40, 2), (43, 1), (46, 1), (61, 2), (64, 1), (65, 1), (74, 2), (79, 1), (84, 2), (86, 2), (100, 1), (104, 1), (111, 1), (117, 2), (123, 3), (125, 1), (126, 1), (129, 1), (132, 3), (134, 7), (137, 1), (138, 1), (141, 3), (154, 4), (164, 1), (165, 1), (172, 3), (175, 1), (176, 2), (183, 1), (198, 1), (214, 3), (217, 1), (222, 1), (228, 1), (233, 1), (253, 1), (255, 2), (262, 2), (270, 1), (273, 2), (275, 2), (281, 1), (288, 2), (292, 3), (303, 1), (321, 1), (324, 1), (329, 1), (339, 2), (342, 1), (356, 6), (357, 1), (376, 1), (381, 1), (385, 1), (391, 1), (392, 1), (395, 2), (399, 1), (404, 6), (409, 1), (419, 4), (423, 4), (438, 2), (459, 1), (461, 1), (464, 2), (467, 1), (490, 1), (491, 2), (500, 4), (511, 24), (518, 3), (519, 1), (525, 2), (526, 1), (529, 1), (533, 1), (535, 1), (539, 2), (548, 3), (554, 3), (561, 1), (564, 1), (578, 1), (595, 1), (617, 1), (627, 1), (631, 1), (654, 1), (665, 1), (668

In [7]:
# Training LDA model
from gensim.models.ldamodel import LdaModel as Lda

ldamodel = Lda(doc_term_matrix, num_topics=15, id2word = dictionary)

# Showing the 15 identified topics after the model is trained, where top 10 key terms are listed for each topic
for topic in ldamodel.print_topics(num_topics=15, num_words=10):
    print(topic[0]+1, " ", topic[1],"\n")

1   0.010*"line" + 0.008*"coordinate" + 0.008*"solution" + 0.007*"equations" + 0.006*"row" + 0.005*"equation" + 0.005*"plane" + 0.004*"methods" + 0.004*"cordic" + 0.003*"work" 

2   0.006*"transformation" + 0.006*"equations" + 0.006*"matrices" + 0.005*"solution" + 0.005*"line" + 0.004*"row" + 0.004*"plane" + 0.004*"algorithm" + 0.004*"product" + 0.003*"coordinate" 

3   0.007*"methods" + 0.007*"ring" + 0.006*"product" + 0.006*"element" + 0.005*"sequence" + 0.005*"zero" + 0.005*"vectors" + 0.004*"map" + 0.004*"sum" + 0.003*"equations" 

4   0.016*"group" + 0.009*"coordinate" + 0.006*"product" + 0.005*"rotation" + 0.005*"image" + 0.005*"sum" + 0.004*"line" + 0.004*"three" + 0.004*"euclidean" + 0.004*"matrices" 

5   0.010*"product" + 0.010*"row" + 0.008*"polynomial" + 0.007*"model" + 0.006*"equations" + 0.006*"matrices" + 0.005*"zero" + 0.005*"vectors" + 0.004*"equation" + 0.004*"algorithm" 

6   0.009*"matrices" + 0.009*"product" + 0.006*"vectors" + 0.006*"equations" + 0.005*"inverse" +

In [8]:
# Clustering documents based on topics extracted from LDA model 
from operator import itemgetter
def cluster(doc_term_matrix, num):
    doc_topics = ldamodel.get_document_topics(doc_term_matrix, minimum_probability=0.20)
    result = [[] for i in range(num)]
    for k,topic in enumerate(doc_topics):
        # Some articles do not have a topic
        if topic:
            topic.sort(key = itemgetter(1), reverse=True)
            result[topic[0][0]].append(k)
    for k in range(len(result)):
        print('Articles(ID) in Cluster ' + str(k+1) + ': ' + ', '.join(map(str, result[k])))
        print()
    return result
cluster_result = cluster(doc_term_matrix, 15)

Articles(ID) in Cluster 1: 26, 64, 78, 126, 139, 144, 165, 167, 179, 195, 203, 205, 219, 227, 265, 268, 291, 305, 306, 325, 336, 345, 378, 409, 420, 427, 434, 439, 448, 454, 503, 504, 535, 541, 561, 564, 567, 639, 644, 673, 676, 686

Articles(ID) in Cluster 2: 28, 52, 85, 109, 214, 277, 283, 331, 344, 350, 384, 412, 431, 442, 444, 479, 489, 516, 518, 601, 652, 677, 682, 684, 692

Articles(ID) in Cluster 3: 25, 36, 39, 80, 100, 116, 118, 122, 123, 129, 166, 184, 196, 199, 209, 226, 229, 234, 246, 278, 287, 300, 318, 333, 353, 362, 373, 398, 411, 435, 437, 443, 456, 459, 460, 482, 486, 500, 522, 532, 537, 556, 589, 597, 598, 604, 622, 624, 632, 635, 640, 664, 668

Articles(ID) in Cluster 4: 2, 17, 21, 30, 32, 53, 55, 68, 91, 95, 110, 138, 145, 194, 211, 220, 239, 241, 249, 261, 319, 328, 330, 332, 355, 368, 369, 380, 382, 391, 393, 405, 406, 417, 418, 462, 466, 468, 469, 485, 507, 514, 521, 530, 542, 552, 566, 617, 649, 660, 665, 672

Articles(ID) in Cluster 5: 8, 12, 16, 18, 24, 48, 51,

In [9]:
# Showing the exact document titles in each cluster
for k in range(len(cluster_result)):
    print('Articles in Cluster ' + str(k+1) + ': ' + ', '.join(map(lambda x: titles[x], cluster_result[k])))
    print()

Articles in Cluster 1: /wiki/Linear algebra, Interval propagation, Trajectory (fluid mechanics), Hundred-dollar, Hundred-digit Challenge problems, Iterative method, Loss of significance, Local convergence, Signal-flow graph, The Nine Chapters on the Mathematical Art, Fangcheng (mathematics), System of linear equations, Balanced set, Plane (geometry), List of finite element software packages, Big M method, S-procedure, System of linear equations, Significant figures, Template:Linear algebra, System of linear equations, Residual (numerical analysis), Linear inequality, Intersection (Euclidean geometry), Van Wijngaarden transformation, Sedrakyan's inequality, System of linear equations, Portal:Linear algebra, Rotation of axes, Zassenhaus algorithm, Three-dimensional space, Telegraphy, Flat (geometry), Line–line intersection, Approximation error, System of linear equations, Line (geometry), Scarborough criterion, Translation of axes, Series acceleration, Superconvergence, Portal:Linear alg

In [10]:
term_topics = ldamodel.get_term_topics('convex', minimum_probability=0.000001)
print(term_topics)

[(0, 0.0007637082), (1, 0.00036834157), (2, 0.000533453), (3, 0.0009955846), (4, 0.000613399), (5, 0.0006990954), (6, 0.0002917314), (7, 0.00051765045), (8, 0.0011452276), (9, 0.00017415907), (10, 0.00023260496), (11, 0.00040177695), (12, 0.0005660611), (13, 0.00025470392), (14, 0.00014823505)]


In [17]:
# Getting related documents based on a term 
def get_related_documents(term, top, doc_term_matrix):
    print('------- Top', top, 'articles related to',term,'-------')
    related_docs = []
    doc_topics = ldamodel.get_document_topics(doc_term_matrix, minimum_probability=0.20)
    term_topics = ldamodel.get_term_topics(term, minimum_probability=0.000001)
    term_topics.sort(key = itemgetter(1), reverse=True)
    for k,topic in enumerate(doc_topics):
        if topic:
            topic.sort(key = itemgetter(1), reverse=True)
            if topic[0][0] == term_topics[0][0]:
                related_docs.append((k,topic[0][1]))
    related_docs.sort(key = itemgetter(1), reverse=True)
    result = []
    for j,doc in enumerate(related_docs):
        print(titles[doc[0]],"\n",doc[1],"\n")   
        result.append(titles[doc[0]])
        if j == top - 1:
            break
related_docs = get_related_documents('input', 7, doc_term_matrix)


------- Top 7 articles related to input -------
Computational complexity 
 0.9984664 

Artificial precision 
 0.9787706 

Semi-infinite programming 
 0.9698835 

Engineering 
 0.9161838 

Symbolic-numeric computation 
 0.91512513 

Projection-valued measure 
 0.8645662 

Orthonormal function system 
 0.8444185 



In [12]:
import pickle 

In [13]:
with open('ldamodel.pkl','wb') as openfile:
    pickle.dump(ldamodel,openfile)

In [15]:
with open('ldamodel.pkl','rb') as f:
    ldamodel = pickle.load(f)

In [16]:
# Getting related documents based on a term 
def get_related_documents(term, top, doc_term_matrix):
    print('------- Top', top, 'articles related to',term,'-------')
    related_docs = []
    doc_topics = ldamodel.get_document_topics(doc_term_matrix, minimum_probability=0.20)
    term_topics = ldamodel.get_term_topics(term, minimum_probability=0.000001)
    term_topics.sort(key = itemgetter(1), reverse=True)
    for k,topic in enumerate(doc_topics):
        if topic:
            topic.sort(key = itemgetter(1), reverse=True)
            if topic[0][0] == term_topics[0][0]:
                related_docs.append((k,topic[0][1]))
    related_docs.sort(key = itemgetter(1), reverse=True)
    result = []
    for j,doc in enumerate(related_docs):
        print(titles[doc[0]],"\n",doc[1],"\n")   
        result.append(titles[doc[0]])
        if j == top - 1:
            break
related_docs = get_related_documents('convex', 7, doc_term_matrix)

------- Top 7 articles related to convex -------
Frame (linear algebra) 
 0.9966995 

Convex cone 
 0.9965028 

Von Neumann stability analysis 
 0.9950839 

Gradient discretisation method 
 0.99346876 

Linear approximation 
 0.99240863 

James H. Wilkinson 
 0.9911048 

James H. Wilkinson 
 0.9911048 



In [138]:
def get_theme(doc, cluster_result):
    doc_id = titles.index(doc)
    if doc_id == -1:
        print('Document not found.')
        return
    for i, cluster in enumerate(cluster_result):
        if doc_id in cluster:
            return i+1
    return 0
cluster_num = get_theme('Absolutely convex set', cluster_result)
print(cluster_num)

6


In [196]:
# Implementing tf-idf model; the only information needed from the previous part is the doc_term_matrix
from gensim.models import TfidfModel, LsiModel
tfidf_model = TfidfModel(doc_term_matrix, dictionary = dictionary)
print(tfidf_model)
vector = tfidf_model[doc_term_matrix[0]]
print(vector[0])


TfidfModel(num_docs=694, num_nnz=169581)
(0, 0.045342656413852538)


In [150]:
# Implementing LSI model; the only information needed from the previous part is the doc_term_matrix
lsi_model = LsiModel(doc_term_matrix, id2word=dictionary)
print(lsi_model)

LsiModel(num_terms=6533, num_topics=200, decay=1.0, chunksize=20000)


In [171]:
# Creating the similarity matrix from simple bag-of-words model (# of documents * # of documents)
from gensim import similarities

index = similarities.MatrixSimilarity(doc_term_matrix, num_features=len(dictionary))
print(len(index[doc_term_matrix[693]])) # 694 * 694 matrix

694


In [172]:
# Training tf-idf model from bag-of-word dataset
model_tfidf = TfidfModel(doc_term_matrix, id2word=dictionary, normalize=False)

In [187]:
# Applying tf-idf model to all vectors
from gensim.corpora import MmCorpus
MmCorpus.serialize('./corpus_tfidf.mm', model_tfidf[doc_term_matrix], progress_cnt=100)

In [188]:
corpus_tfidf = MmCorpus('./corpus_tfidf.mm') # Loading back the corpus file after applying tf-idf
model_lsi = LsiModel(corpus_tfidf, num_topics=15, id2word=dictionary)
# Applying LSI model to all vectors
index = similarities.MatrixSimilarity(model_lsi[corpus_tfidf], num_features=len(dictionary))
print(index)
index.save('./lsi_index.mm') # Saving the similarity matrix to a local matrix market file named './lsi_model.mm'

MatrixSimilarity<694 docs, 6533 features>


In [195]:
# Loading the similarity matrix back from the local file
similarity_matrix = similarities.MatrixSimilarity.load('./lsi_index.mm')
print(len(similarity_matrix))

694
